In [2]:
import json
import os
from pprint import pprint

In [3]:
def evaluate(output_config_name=None, on_train_or_test=None, write_result=True, checkpoint=0, full_name=None):
    if write_result:
        with open(os.path.join("./output_config", output_config_name+".json")) as f:
            config = json.load(f)
    if full_name == None:
        if checkpoint == 0:
            llm_result_path = f"./LLaMA-Factory/saves/Baichuan2-13B-Chat/lora/predict/gnn_aggregator_{output_config_name}_predict_on_{on_train_or_test}/generated_predictions.jsonl"
        else:
            llm_result_path = f"./LLaMA-Factory/saves/Baichuan2-13B-Chat/lora/predict/gnn_aggregator_{output_config_name}_predict_on_{on_train_or_test}/checkpoint-{checkpoint}/generated_predictions.jsonl"
    else:
        llm_result_path = f"./LLaMA-Factory/saves/Baichuan2-13B-Chat/lora/predict/{full_name}/generated_predictions.jsonl"
    result = []
    total = 0
    with open(llm_result_path) as f:
        while True:
            s = f.readline()
            if s == "":
                break
            else:
                total += 1
            try:
                j = json.loads(s)
                j["label"] = j["label"]
                j["predict"] = j["predict"]
                # j["label"] = int(j["label"][-2])
                # j["predict"] = int(j["predict"][-2])
                result.append(j)
            except:
                pass
    # print(result)

    same = 0
    for i, r in enumerate(result):
        if r["label"] == r["predict"]:
            same += 1
    print(f"total {total}, same {same}, simularity {same / total}")

    if write_result:
        config[on_train_or_test + "_acc" + ("" if checkpoint==0 else ("_"+str(checkpoint)))] = same / total
        with open(os.path.join("./output_config", output_config_name+".json"), "w") as f:
            json.dump(config, f, indent=4)

    
    return "{:.4f}".format(same / total)

In [ ]:
for m in ["gcn_", "gat_", "gin_", "gcn_gat_gin"]:
    for d0 in ["arxiv", "products"]:
        for d1 in ["arxiv", "pubmed", "cora"]:
            for shot in ["0_shot", "1_shot", "5_shot"]:
                for ri in r:
                    if ri[0]==m and ri[1] == d0 and ri[2] == d1 and ri[3] == shot:
                        print(ri[4], end=" & ")
    print()

In [ ]:
for root, dirs, files in os.walk("./output_config"):
    for file in files:
        if "ensemble" in file:
            output_config_name = file[:-5]
            for checkpoint in [0]:
                for on_train_or_test in ["train", "test"]:
                    if checkpoint == 0:
                        t_dir = f"./LLaMA-Factory/saves/Baichuan2-13B-Chat/lora/predict/gnn_aggregator_{output_config_name}_predict_on_{on_train_or_test}/generated_predictions.jsonl"
                    else:
                        t_dir = f"./LLaMA-Factory/saves/Baichuan2-13B-Chat/lora/predict/gnn_aggregator_{output_config_name}_predict_on_{on_train_or_test}/checkpoint-{checkpoint}/generated_predictions.jsonl"
                    if os.path.exists(t_dir):
                        print(output_config_name)
                        evaluate(output_config_name, on_train_or_test, checkpoint=checkpoint)
                    else:
                        pass
                        # print(f"Warning: Cannot find {t_dir}")